## Exercício - Titanic
disponível em https://www.kaggle.com/c/titanic/overview

#### Descrição
O naufrágio do RMS Titanic é um dos naufrágios mais famosos da história. Em 15 de abril de 1912, durante sua viagem inaugural, o Titanic afundou após colidir com um iceberg, matando 1502 de 2224 passageiros e tripulantes. Essa tragédia chocou a comunidade internacional e levou a melhores regulamentações de segurança para os navios.

Uma das razões pelas quais o naufrágio levou a tal perda de vidas foi que não havia botes salva-vidas suficientes para os passageiros e a tripulação. Embora houvesse algum elemento de sorte envolvido na sobrevivência do naufrágio, alguns grupos de pessoas eram mais propensos a sobreviver do que outros, como mulheres, crianças e a classe alta.

Nesse desafio, a atividade é que você conclua a análise de que tipo de pessoa provavelmente sobreviverá. Em particular, solicitamos você deve aplicar classificadores de árvores de decisão para esta atividade.


### Descrição do dataset
Os dados foram divididos em dois grupos:

conjunto de treinamento (train.csv)
conjunto de teste (test.csv)
O conjunto de treinamento deve ser usado para criar seus modelos de aprendizado de máquina. Para o conjunto de treinamento, fornecemos o resultado (também conhecido como "verdade do solo") para cada passageiro. Seu modelo será baseado em "recursos", como sexo e classe dos passageiros. Você também pode usar a engenharia de recursos para criar novos recursos.

O conjunto de teste deve ser usado para ver o desempenho do seu modelo em dados não vistos. Para o conjunto de testes, não fornecemos a verdade básica para cada passageiro. É seu trabalho prever esses resultados. Para cada passageiro no conjunto de testes, use o modelo que você treinou para prever se eles sobreviveram ou não ao naufrágio do Titanic.

Também incluímos gender_submission.csv, um conjunto de previsões que supõem que todas e apenas passageiras sobrevivem, como um exemplo da aparência de um arquivo de envio.

### Dicionário de Dados

**Variable -	Definition	- Key**

survival - Survival	- 0 = No, 1 = Yes

pclass -	Ticket class -	1 = 1st, 2 = 2nd, 3 = 3rd

sex -	Sex	

Age	- Age in years	

sibsp -	# of siblings / spouses aboard the Titanic	

parch- # of parents / children aboard the Titanic	

ticket - Ticket number	

fare - Passenger fare	

cabin- Cabin number	

embarked- Port of Embarkation	C = Cherbourg, Q = Queenstown, S = Southampton

In [298]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.model_selection import KFold
from sklearn.svm import SVR
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

In [299]:
treino = pd.read_csv('train.csv')
teste = pd.read_csv('test.csv')
y = treino['Survived']
treino.drop(['PassengerId','Name','Survived','Ticket'],axis=1,inplace=True)
X = treino
y_teste = pd.read_csv('gender_submission.csv')
y_teste.drop(['PassengerId'], axis=1, inplace=True)
print(y_teste)
#y_teste = teste['Survived']
teste.drop(['PassengerId','Name','Ticket'],axis=1,inplace=True)
X_teste = teste

     Survived
0           0
1           1
2           0
3           0
4           1
5           0
6           1
7           0
8           1
9           0
10          0
11          0
12          1
13          0
14          1
15          1
16          0
17          0
18          1
19          1
20          0
21          0
22          1
23          0
24          1
25          0
26          1
27          0
28          0
29          0
..        ...
388         0
389         0
390         0
391         1
392         0
393         0
394         0
395         1
396         0
397         1
398         0
399         0
400         1
401         0
402         1
403         0
404         0
405         0
406         0
407         0
408         1
409         1
410         1
411         1
412         1
413         0
414         1
415         0
416         0
417         0

[418 rows x 1 columns]


In [300]:
X['Cabin'] = X['Cabin'].fillna('Z')
X
X_teste['Cabin'] = X_teste['Cabin'].fillna('Z')
X_teste

,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,3,male,34.5,0,0,7.8292,Z,Q
1,3,female,47.0,1,0,7.0000,Z,S
2,2,male,62.0,0,0,9.6875,Z,Q
3,3,male,27.0,0,0,8.6625,Z,S
4,3,female,22.0,1,1,12.2875,Z,S
5,3,male,14.0,0,0,9.2250,Z,S
6,3,female,30.0,0,0,7.6292,Z,Q
7,2,male,26.0,1,1,29.0000,Z,S
8,3,female,18.0,0,0,7.2292,Z,C
9,3,male,21.0,2,0,24.1500,Z,S


In [301]:
#X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=42, stratify=y)

In [302]:
#X_train['Cabin'] = X_train['Cabin'].apply(lambda x: x[0])
#print(X_train)
X['Cabin'] = X['Cabin'].apply(lambda x: x[0])
print(X)
X_teste['Cabin'] = X_teste['Cabin'].apply(lambda x: x[0])
print(X_teste)

     Pclass     Sex   Age  SibSp  Parch      Fare Cabin Embarked
0         3    male  22.0      1      0    7.2500     Z        S
1         1  female  38.0      1      0   71.2833     C        C
2         3  female  26.0      0      0    7.9250     Z        S
3         1  female  35.0      1      0   53.1000     C        S
4         3    male  35.0      0      0    8.0500     Z        S
5         3    male   NaN      0      0    8.4583     Z        Q
6         1    male  54.0      0      0   51.8625     E        S
7         3    male   2.0      3      1   21.0750     Z        S
8         3  female  27.0      0      2   11.1333     Z        S
9         2  female  14.0      1      0   30.0708     Z        C
10        3  female   4.0      1      1   16.7000     G        S
11        1  female  58.0      0      0   26.5500     C        S
12        3    male  20.0      0      0    8.0500     Z        S
13        3    male  39.0      1      5   31.2750     Z        S
14        3  female  14.0

In [303]:
#X_train['Cabin'] = X_train['Cabin'].replace(['T','G'],'Z', inplace=True)
#print(X_train)

In [304]:
numerical = ['Fare','Age']
imputer_num = SimpleImputer(strategy='mean')
#X_train[numerical] = imputer_num.fit_transform(X_train[numerical])
X[numerical] = imputer_num.fit_transform(X[numerical])

numerical = ['Fare','Age']
imputer_num = SimpleImputer(strategy='mean')
#X_train[numerical] = imputer_num.fit_transform(X_train[numerical])
X_teste[numerical] = imputer_num.fit_transform(X_teste[numerical])
print(X_teste)


     Pclass     Sex       Age  SibSp  Parch      Fare Cabin Embarked
0         3    male  34.50000      0      0    7.8292     Z        Q
1         3  female  47.00000      1      0    7.0000     Z        S
2         2    male  62.00000      0      0    9.6875     Z        Q
3         3    male  27.00000      0      0    8.6625     Z        S
4         3  female  22.00000      1      1   12.2875     Z        S
5         3    male  14.00000      0      0    9.2250     Z        S
6         3  female  30.00000      0      0    7.6292     Z        Q
7         2    male  26.00000      1      1   29.0000     Z        S
8         3  female  18.00000      0      0    7.2292     Z        C
9         3    male  21.00000      2      0   24.1500     Z        S
10        3    male  30.27259      0      0    7.8958     Z        S
11        1    male  46.00000      0      0   26.0000     Z        S
12        1  female  23.00000      1      0   82.2667     B        S
13        2    male  63.00000     

In [305]:
categorical=['Pclass', 'Sex','SibSp','Parch']
imputer_cat = SimpleImputer(strategy='most_frequent')
#X_train[categorical] = imputer_cat.fit_transform(X_train[categorical])
#X_train = pd.get_dummies(X_train, drop_first=True)
#X_test = pd.get_dummies(X_test, drop_first=True)
X[categorical] = imputer_cat.fit_transform(X[categorical])
X['Sex'] = pd.get_dummies(X['Sex'], drop_first=True)
X['Cabin'] = pd.get_dummies(X['Cabin'], drop_first=True)
X['Embarked'] = pd.get_dummies(X['Embarked'], drop_first=True)
#X_test = pd.get_dummies(X_test, drop_first=True)

categorical=['Pclass', 'Sex','SibSp','Parch']
imputer_cat = SimpleImputer(strategy='most_frequent')
#X_train[categorical] = imputer_cat.fit_transform(X_train[categorical])
#X_train = pd.get_dummies(X_train, drop_first=True)
#X_test = pd.get_dummies(X_test, drop_first=True)
X_teste[categorical] = imputer_cat.fit_transform(X_teste[categorical])
X_teste['Sex'] = pd.get_dummies(X_teste['Sex'], drop_first=True)
X_teste['Cabin'] = pd.get_dummies(X_teste['Cabin'], drop_first=True)
X_teste['Embarked'] = pd.get_dummies(X_teste['Embarked'], drop_first=True)
#X_teste = pd.get_dummies(X_teste, drop_first=True)
#X_test = pd.get_dummies(X_test, drop_first=True)

In [306]:
print(X_teste)

     Pclass  Sex       Age  SibSp  Parch      Fare  Cabin  Embarked
0         3    1  34.50000      0      0    7.8292      0         1
1         3    0  47.00000      1      0    7.0000      0         0
2         2    1  62.00000      0      0    9.6875      0         1
3         3    1  27.00000      0      0    8.6625      0         0
4         3    0  22.00000      1      1   12.2875      0         0
5         3    1  14.00000      0      0    9.2250      0         0
6         3    0  30.00000      0      0    7.6292      0         1
7         2    1  26.00000      1      1   29.0000      0         0
8         3    0  18.00000      0      0    7.2292      0         0
9         3    1  21.00000      2      0   24.1500      0         0
10        3    1  30.27259      0      0    7.8958      0         0
11        1    1  46.00000      0      0   26.0000      0         0
12        1    0  23.00000      1      0   82.2667      1         0
13        2    1  63.00000      1      0   26.00

In [309]:
scores = []
dtc =  DecisionTreeClassifier(max_depth=5, random_state=42)
X = np.array(X)
y = np.array(y)
cv = KFold(n_splits=10, random_state=42, shuffle=False)
for train_index, test_index in cv.split(X):
    #print("Train Index: ", train_index, "\n")
    #print("Test Index: ", test_index)
    X_train, X_test, y_train, y_test = X[train_index], X[test_index], y[train_index], y[test_index]
    dtc.fit(X_train, y_train)
    scores.append(dtc.score(X_test, y_test))

In [310]:
y_pred = dtc.predict(X_teste)
print(y_pred)
print(y_teste)
accuracy_score(y_teste, y_pred)

[0 0 0 0 1 0 1 0 1 0 0 0 1 0 1 1 0 0 1 0 0 1 1 0 1 0 1 0 0 0 0 0 1 1 0 0 1
 1 0 1 0 0 0 1 1 0 0 0 1 1 0 0 1 1 0 0 0 0 0 1 0 0 0 1 1 1 1 0 0 1 1 0 1 0
 1 0 0 1 0 1 1 0 0 0 0 0 1 1 1 1 1 0 1 0 0 0 1 0 1 0 1 0 0 0 1 0 0 0 0 0 0
 1 1 1 1 0 0 1 0 1 1 0 1 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0
 0 0 1 0 0 1 0 0 1 1 0 1 1 1 1 0 0 1 0 0 1 1 0 0 0 0 0 1 1 0 1 1 0 0 1 0 1
 0 1 0 0 0 0 0 0 0 1 0 1 1 0 1 1 1 0 1 0 0 1 0 1 0 0 0 0 1 0 0 1 0 1 0 1 0
 1 0 1 1 0 1 0 0 0 1 0 0 0 0 0 0 1 1 1 1 0 0 0 0 1 0 1 1 1 0 0 0 0 0 0 0 1
 0 0 0 1 1 0 0 0 0 1 0 0 0 1 1 0 1 0 0 0 0 1 1 1 1 1 0 0 0 0 0 0 1 0 0 0 0
 1 0 0 0 0 0 0 0 1 1 0 1 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 1 0 1 0 0 0 1 0 0
 1 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 1 1 0 0 0 1 0 1 0 0 1 0 1 1 0 1 0 0 0 1 0
 0 1 0 0 1 1 1 0 0 0 0 0 1 1 0 1 0 0 0 0 0 1 0 0 0 1 0 1 0 0 1 0 1 1 0 0 0
 0 1 1 1 1 1 0 1 0 0 0]
     Survived
0           0
1           1
2           0
3           0
4           1
5           0
6           1
7           0
8           1
9          

0.9449760765550239